In [4]:
# Import libraries and dependencies
import pandas as pd
from pathlib import Path
import seaborn as sns

In [ ]:
##Pseudo Code for Analysis
#Read in APIs
#Read CSVs
#All Dates by year

In [3]:
#Pct change by Zip
#Price Per Square foot by Zip
    #"Date" "Zip" "Price Per Square foot"
#Schools
    #"Date" "Zip" "School District" "Grade"
#Crime
    #"Date" "Zip" "Type" 
#Income Tax
    #"Date" "Zip" "Income Tax Paid" "Number of returns" Total Charitable Contributions" "Brackets"
#Real Estate Tax
    #"Date" "Zip" "Real Estate taxes paid" "personal property tax"
#Opportunity Zones
    #"Zip"
#Stray Animals
    #"Date" "Zip" "Number of Strays Reported"
#Population
    #"Date" "Zip" "Number of People"
#Construction Permits
    #"date" "zip" "permit_count" "permit_type" (Residential vs Commercial)
#Restaurant, Bars, & Clubs

In [ ]:
#### Work In Progress
#### correlate_construction_permits
# def correlate_construction_permits(permits_df, pct_chg_df, groupby_month=bool):
#     # input: "date" "zip" "permit_count" "permit_type" (Residential vs Commercial)
#     # input: pct_chg_df with date, zip, pct_change
#     # return dataframe with date, zip, correlation_residential, correlation_commercial
    
#     # group permits_df by frequency
#     if groupby_month == True:
#         # group by yearh and month
#         permits_by_type_df = permits_df.groupby([index.month, index.year])['Type'].sum()
#     elif groupby_month == False:
#         # group only by year
#         permits_by_type_df = permits_df.groupby([index.year])['Type'].sum()
    
    
    
#     combined_df = pd.concat([permits_df, pct_chg_df], axis="columns", join="inner")
#     # group by 

In [68]:
# Read in the construction permits data
permits_data = Path("../resources/dan/sample_permits_dan.csv")
permits = pd.read_csv(construction_permits_data, sep='\t')

# set 'date' column type to date with year and month only format
permits['date'] = pd.to_datetime(construction_permits['date'], format= '%Y%m')

#  groupby zip and date then calculate sum of permit_count for each zip/date grouping
permits_by_zipdate = construction_permits.groupby(['zip','date']).sum()

# spot check data
permits_by_zipdate.head()


permit_count
zip   date                    
77002 2016-01-01           771
      2016-02-01           618
      2016-03-01           306
      2016-04-01           725
      2016-05-01           507

In [73]:
# Read in the pct_change data
pct_chg_data = Path("../resources/dan/sample_pct_chg_dan.csv")
pct_chg = pd.read_csv(pct_chg_data, sep='\t')

# set 'date' column type to date with year and month only format
pct_chg['date'] = pd.to_datetime(pct_chg['date'], format= '%Y%m')

# groupby zip and date then calculate sum of permit_count for each zip/date grouping
pct_chg_by_zipdate = pct_chg.groupby(['zip','date']).sum()

# spot check data
pct_chg_by_zipdate.head()


pct_chg
zip   date               
77002 2016-01-01     0.07
      2016-02-01     0.09
      2016-03-01    -0.02
      2016-04-01     0.01
      2016-05-01     0.02

In [226]:
#  combine permits_by_zipdate and pct_chg_by_zipdate by matching indexes
combined_permits_pct_chg_df = pd.concat([permits_by_zipdate, pct_chg_by_zipdate], axis="columns", join="inner")

# drop any null data rows
combined_permits_pct_chg_df.dropna(inplace=True)

permit_count  pct_chg
zip   date                             
77002 2016-01-01           771     0.07
      2016-02-01           618     0.09
      2016-03-01           306    -0.02
      2016-04-01           725     0.01
      2016-05-01           507     0.02
...                        ...      ...
77004 2020-02-01           513    -0.03
      2020-03-01           597    -0.03
      2020-04-01           258     0.04
      2020-05-01           464     0.09
      2020-06-01           360     0.02

[162 rows x 2 columns]

In [224]:

# Set column names for use in initializing dataframe, for appending rows, and for resetting index
cols = ['zip','permit_correlation']

#initialize correlation_per_zip dataframe
correlation_per_zip = pd.DataFrame(columns=cols)

# iterate through each zip
for zip, df in combined_permits_pct_chg_df.groupby(level=0):

    # calculate correlation of this zip's permit count to pct_chg
    correlation_df = df.corr()
    
    # record specific correlation value from correlation dataframe
    correlation_value = correlation_df.loc['permit_count']['pct_chg']
    
    # create row/series based on this zip's correlation to append to  dataframe
    correlation_row = [{cols[0] : zip, cols[1] : correlation_value}]
    
    # append row to dataframe
    correlation_per_zip = correlation_per_zip.append(correlation_row)

# reset  dataframe index to zip/cols[0]
correlation_per_zip.set_index(cols[0])
 


,permit_correlation
zip,
77002,0.199488
77003,-0.091708
77004,0.025493
